In [1]:
import os
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import shutil as sh
import re
import os
import matplotlib as mpl
from matplotlib.colors import Normalize 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

In [2]:
class calc():
    def __init__(self):
        self.R3_list=[np.round(r,1) for r in np.linspace(-4.0,4.0,81)]
        self.path='C:\\Users\\koyama\\OSC\\interaction\\tetracene\\'
        self.A1_list=[int(A1) for A1 in np.linspace(1,40,40)]
        self.N=41

    def calculator(self,path_file):
        with open(path_file,'r') as f:
            lines=f.readlines()
        lines_E=[]
        for line in lines:
            if line.find('E(RB3LYP)')>-1:
                lines_E.append(float(line.split()[4])*627.510)
        E_list=[lines_E[5*i]-lines_E[5*i+1]-lines_E[5*i+2] for i in range(int(len(lines_E)/5))]
        return E_list
        
    def calc_step1(self):
        df=pd.DataFrame(columns=['R1','R2','heri','E_t','E_p','E'])
        out_dir=self.path+'step1'#outファイルのあるディレクトリ
        for file in os.listdir(out_dir):
            if file[-4:]!='.out' and file[-4:]!='.log':
                continue
            _,R1,_,R2,_,theta=re.split('[_=]', file[:-4])
            E_list=self.calculator(out_dir+'\\'+file)
            if len(E_list)!=2:
                print(file)
                continue
            E_t=float(E_list[0]);E_p=float(E_list[1])
            data=pd.Series([R1,R2,theta,E_t,E_p,4*E_t+2*E_p],index=df.columns)#セル当たりの格子エネルギー
            df=df.append(data,ignore_index=True)
        df=df.sort_values('heri')
        df=df.reset_index(drop=True)
        df.to_csv(out_dir+'\\step1.csv',index=False)

        
    def calc_step2(self):
        df=pd.DataFrame(columns=['R3t','R3p','Et1','Ep','Et2','E'])
        Et_list=self.calculator(self.path+'step2/t.log')
        Ep_list=self.calculator(self.path+'step2/p.log')
        for id_t,R3t in enumerate(self.R3_list):
            for id_p,R3p in enumerate(self.R3_list):
                if (R3t-R3p)>4.0 or (R3t-R3p)<-4.0:
                    continue
                Et2 = Et_list[id_t-id_p+40]
                Et1 = Et_list[id_t]
                Ep = Ep_list[id_p]
                data = pd.Series([R3t,R3p,Et1,Ep,Et2,2*Et1+2*Et2+2*Ep],index=df.columns)
                df = df.append(data,ignore_index=True)
        df.to_csv(self.path+'step2/step2.csv',index=False)
        
    def calc_step3(self):
        #Z=1か2かで分ける。
        df_1=pd.DataFrame(columns=['R1','R2','heri','R3t','R3p','Rai0','Rbi0','R3i0','E','i0','ip1','it1','it2','ip2','it3','it4'])
        df_2=pd.DataFrame(columns=['R1','R2','heri','R3t','R3p','Rai0','Rbi0','R3i0','E','i0','ip1','it1','it2','ip2','it3','it4'])
        path_csv=self.path+'step3/'
        for file in os.listdir(path_csv):
            if file[-4:]!='.log':
                continue
            _,R1,_,R2,_,heri,_,R3t,_,R3p,_,Rai0,_,Rbi0,_,R3i0,_,Z=re.split('[=_]',file[:-4])
            E_list=self.calculator(path_csv+file)
            para_list=list(map(float,[R1,R2,heri,R3t,R3p,Rai0,Rbi0,R3i0]))
            if len(E_list)!=7:
                print(para_list)
            data=pd.Series(para_list+[sum(E_list)]+E_list,index=df_1.columns)#Z=1,2で同じ操作。データ格納
            if int(Z)==1:
                df_1=df_1.append(data,ignore_index=True)
            elif int(Z)==2:
                df_2=df_2.append(data,ignore_index=True)
        #生データdf_1,df_2をcsv出力
        df_raw=pd.concat([df_1,df_2],ignore_index=True);df_raw.to_csv(path_csv+'raw_data.csv')
        
        #df_2のパラメータをZ=1へ変換してマージ。E_sumだけを出力
        df_2['R3t']=df_2['R3t'].values-df_2['R3p'].values#R3t=R3t2'=R3t'-R3p'
        df_2['R3p']=-df_2['R3p'].values#R3_p=-R3_p'
        df_2['Rbi0']=-df_2['Rbi0'].values#Rb_i0:-Rb_i0'
        
        self.df_1=df_1;self.df_2=df_2
        df=pd.merge(df_1.round(6).loc[:,'R1':'E'],df_2.round(6).loc[:,'R1':'E'],on=['R1','R2','heri','R3t','R3p','Rai0','Rbi0','R3i0'],suffixes=['_Z=1','_Z=2'])
        assert df.shape[0]!=0
        df['E_total']=df['E_Z=1'].values+df['E_Z=2'].values
        df.to_csv(path_csv+'step3.csv',index=False)
        
    def calc_step2B3(self):
        df=pd.DataFrame(columns=['A2','a','b','Et','Ep','Ei0','Eip1','Eit1','Eip2','Eit3','E'])
        out_dir=self.path+'step2B3'#outファイルのあるディレクトリ
        for file in os.listdir(out_dir):
            if file[-4:]!='.out' and file[-4:]!='.log':
                continue
            _,A2,_,a,_,b=re.split('[_=]', file[:-4])
            E_list=self.calculator(out_dir+'\\'+file)
            if len(E_list)!=7:
                print(file)
            Et,Ep,Ei0,Eip1,Eit1,Eip2,Eit3=list(map(float,E_list))
            E_6inter=4*Et+2*Ep
            E_interlayer=2*(Ei0+Eip1+2*Eit1+Eip2+2*Eit3)
            data=pd.Series([A2,a,b,Et,Ep,Ei0,Eip1,Eit1,Eip2,Eit3,E_6inter+E_interlayer],index=df.columns)#セル当たりの格子エネルギー
            df=df.append(data,ignore_index=True)
        df.to_csv(out_dir+'\\step2B3.csv',index=False)


# step1

In [3]:
calc_1=calc()
calc_1.calc_step1()

# step2

In [3]:
calc_2=calc()
calc_2.calc_step2()

# step2B+3

In [8]:
calc_2B3=calc()
calc_2B3.calc_step2B3()

# step3

In [3]:
calc_3=calc()
calc_3.calc_step3()